In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rabieelkharoua/chronic-kidney-disease-dataset-analysis")

print("Path to dataset files:", path)

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/morning/.cache/kagglehub/datasets/rabieelkharoua/chronic-kidney-disease-dataset-analysis/versions/1


In [2]:
import os
os.listdir(path)

['Chronic_Kidney_Dsease_data.csv']

In [3]:
import pandas as pd
import fairtl as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [4]:
df = pd.read_csv(f'{path}/Chronic_Kidney_Dsease_data.csv')
df.head()

,PatientID,Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,...,Itching,QualityOfLifeScore,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,Diagnosis,DoctorInCharge
0,1,71,0,0,0,2,31.069414,1,5.128112,1.676220,...,7.556302,76.076800,0,0,1,1.018824,4.966808,9.871449,1,Confidential
1,2,34,0,0,1,3,29.692119,1,18.609552,8.377574,...,6.836766,40.128498,0,0,0,3.923538,8.189275,7.161765,1,Confidential
2,3,80,1,1,0,1,37.394822,1,11.882429,9.607401,...,2.144722,92.872842,0,1,1,1.429906,7.624028,7.354632,1,Confidential
3,4,40,0,2,0,1,31.329680,0,16.020165,0.408871,...,7.077188,90.080321,0,0,0,3.226416,3.282688,6.629587,1,Confidential
4,5,43,0,1,1,2,23.726311,0,7.944146,0.780319,...,3.553118,5.258372,0,0,1,0.285466,3.849498,1.437385,1,Confidential


In [5]:
df.shape

(1659, 54)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1659 entries, 0 to 1658
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   PatientID                      1659 non-null   int64  
 1   Age                            1659 non-null   int64  
 2   Gender                         1659 non-null   int64  
 3   Ethnicity                      1659 non-null   int64  
 4   SocioeconomicStatus            1659 non-null   int64  
 5   EducationLevel                 1659 non-null   int64  
 6   BMI                            1659 non-null   float64
 7   Smoking                        1659 non-null   int64  
 8   AlcoholConsumption             1659 non-null   float64
 9   PhysicalActivity               1659 non-null   float64
 10  DietQuality                    1659 non-null   float64
 11  SleepQuality                   1659 non-null   float64
 12  FamilyHistoryKidneyDisease     1659 non-null   i

In [7]:
nan_count = df.isna().sum().sum()
print(nan_count)

0


In [8]:
df['Gender'].unique()

array([0, 1])

In [9]:
df.drop('DoctorInCharge', inplace=True, axis=1)

In [10]:
y = df['Diagnosis'].values
X = df.drop('Diagnosis', axis=1)

In [11]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(1659, 52)
(1659, 49)


In [12]:
# build mask
gender_0_mask = df['Gender'] == 0
gender_1_mask = df['Gender'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  804
Male:  855


In [13]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [14]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group  SVM_TPR  SVM_TNR  SVM_FPR  SVM_FNR  SVM_TP  SVM_TN  SVM_FP  \
0      1  Female      1.0      0.0      1.0      0.0      75       0       6   
1      2  Female      1.0      0.0      1.0      0.0      74       0       7   
2      3  Female      1.0      0.0      1.0      0.0      76       0       5   
3      4  Female      1.0      0.0      1.0      0.0      73       0       8   
4      5  Female      1.0      0.0      1.0      0.0      74       0       6   
5      6  Female      1.0      0.0      1.0      0.0      73       0       7   
6      7  Female      1.0      0.0      1.0      0.0      68       0      12   
7      8  Female      1.0      0.0      1.0      0.

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
results_df = pd.concat(results_list, ignore_index=True)

In [16]:
result_path = './results/k114_result.xlsx'
results_df.to_excel(result_path, index=False)

In [17]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,1,0,1,0,75,0,6,0,...,5,3,0.986667,0.166667,0.833333,0.013333,74,1,5,1
1,2,Female,1,0,1,0,74,0,7,0,...,5,4,1.000000,0.000000,1.000000,0.000000,74,0,7,0
2,3,Female,1,0,1,0,76,0,5,0,...,5,3,0.934211,0.000000,1.000000,0.065789,71,0,5,5
3,4,Female,1,0,1,0,73,0,8,0,...,7,0,0.972603,0.000000,1.000000,0.027397,71,0,8,2
4,5,Female,1,0,1,0,74,0,6,0,...,3,0,0.972973,0.333333,0.666667,0.027027,72,2,4,2


In [18]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
DT -TPR: MannwhitneyuResult(statistic=np.float64(31.0), pvalue=np.float64(0.1614988122969947))
DT - FPR: TtestResult(statistic=np.float64(-0.1293740600243537), pvalue=np.float64(0.898496378901599), df=np.float64(18.0))
DT - FN/FP: MannwhitneyuResult(statistic=np.float64(58.0), pvalue=np.float64(0.5704604728984778))
RF -TPR: MannwhitneyuResult(statistic=np.float64(38.5), pvalue=np.float64(0.23426628444669173))
RF - FPR: MannwhitneyuResult(statistic=np.float64(20.0), pvalue=np.float64(0.005971963624595338))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(61.5), pvalue=np.float64(0.2339062109885507))
LR -TPR: TtestResult(statistic=np.float64(-0.9772080401841955), pvalue=np.float64(0.3414178677210178), df=np.float64(18.0))
LR - FPR: TtestResul